<a href="https://colab.research.google.com/github/vsolodkyi/NeuralNetworks_SkillBox/blob/main/module_14/%D0%A0%D0%B5%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_LSTM_(%D0%BF%D1%80%D0%BE%D0%B4%D0%BE%D0%BB%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5)_%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Реализация LSTM (продолжение)
В этом уроке мы продолжим знакомство с LSTM и рассмотрим подробнее флаг `return_sequences`

### Используем TensorFlow 2.0

На момент подготовки этих материалов в Google Colab по умолчанию используется версия TensorFlow 1.X

Переключаемся на версию 2.0 (работает только в Colab)

In [ ]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


### Загрузка библиотек
TensorFlow должен иметь как минимум версию 2.0

In [ ]:
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.0.0


### Входной тензор
Подготовим входной тензор для тестов аналогично тому, как мы это делали в предыдущем уроке.

Размерности тензора `x`: (батч, длина цепочки, размер эмбеддинга)

In [ ]:
BATCH_SIZE = 2
SEQ_LEN = 100 # Длина последовательности
EMB_SIZE = 16 # Размер векторного представления (эмбеддинга)

x = np.random.rand(BATCH_SIZE, SEQ_LEN, EMB_SIZE).astype(np.float32)
print(x.shape)

(2, 100, 16)


### Флаг `return_sequences`

В рекуррентных слоях, как правило, есть флаг `return_sequences`. Ранее мы использовали `return_sequences=True`

Если же установить флаг `return_sequences=False`, то рекуррентный слой будет выдавать лишь **последний** вектор `h`, а не всю цепочку векторов `h`.

In [ ]:
H_SIZE = 32
lstm = tf.keras.layers.LSTM(H_SIZE, return_sequences=False, recurrent_activation='sigmoid')

Запускаем инференс (прямое распространение) для созданного слоя и проверяем размерность выходного тензора.

Она должна была получиться `(BATCH_SIZE, H_SIZE)`

In [ ]:
y = lstm(x)
print(y.shape)

(2, 32)


**[Задание 1]** Модернизируйте класс `LSTM` из предыдущего урока, добавив в него (в конструктор) флаг `return_sequences`, чтобы поддерживались обе опции (True/False).

**[Задание 2]** Сравните результаты работы нового класса `LSTM` (с параметром `return_sequences=False`) с исходной моделью на Keras так же, как мы это делали раньше (разница должна получиться равной нулю или очень маленькой)